In [1]:
import pandas as pd
df = pd.read_csv('./pseudo/twitch.csv')
df.head()

,text,none,curse
0,ㅋㅋ,0.993,0.007
1,a를 더 자주누르면되는거 아닌가? ㅋㅋㅋㅋ,0.992,0.008
2,그 누구지 땅우양은 그냥 사거릴 외우라던대,0.992,0.008
3,선생님 두오는 몬가요?,0.992,0.008
4,뭐 자신만의 카이팅이 있으니까..,0.992,0.008


In [2]:
df2 = pd.read_csv('labeled.csv')
df2.head()

,text,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요ㅠ,0
2,"못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지,그래야, 공정한 사회지...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐? 메걸리안들 생각이 없노,1


In [3]:
df = list(df['text'])
df2 = list(df2['text'])

In [4]:
from data import preprocessing
df = preprocessing(df)
df2 = preprocessing(df2)

df = list(df)
df2 = list(df2)

100%|██████████| 19511/19511 [00:00<00:00, 85308.04it/s]


In [11]:
with open('./pseudo/corpus.txt', 'a', encoding='utf-8') as f:
    for text in df:
        if text != text: continue
        f.write(text+'\n')
    for text in df2:
        if text != text: continue
        f.write(text+'\n')

In [12]:
from tokenizers import BertWordPieceTokenizer

# load sentences
tokenizer = BertWordPieceTokenizer(
    vocab=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
    wordpieces_prefix="##",
)

limit_alphabet = 100000
vocab_size = 500000

tokenizer.train(
    files='./pseudo/corpus.txt',
    vocab_size=vocab_size,
    limit_alphabet=limit_alphabet,
    min_frequency=2,
    show_progress=True,
)

tokenizer.save("./pretrained_tokenizer/tokenizer.json", True)  # save tokenizer.json, pretty=True로 두시면 json 형식이 보기좋게 저장됩니다
tokenizer.save_model('./pretrained_tokenizer/')  # save vocab.txt

['./pretrained_tokenizer/vocab.txt']